In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import zip_longest
from currency_converter import CurrencyConverter
from datetime import date

raw_dir = Path('../data/raw/').resolve()

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
def get_df(name):
    df = pd.read_excel(raw_dir.as_posix() + '/{0}'.format(name))
    df.loc[:, 'FECHA_INICIO'] = pd.to_datetime(df.FECHA_INICIO, yearfirst=True)
    df.loc[:, 'FECHA_FIN'] = pd.to_datetime(df.FECHA_FIN, yearfirst=True)
    return df

file_names = [
    '/Procedimiento de Contratación 2010-2012.xlsx',
    '/Procedimiento de Contratación 2013.xlsx',
    '/Procedimiento de Contratación 2014.xlsx',
    '/Procedimiento de Contratación 2015.xlsx',
    '/Procedimiento de Contratación 2016.xlsx'
]

dfs = joblib.Parallel(n_jobs=8)(joblib.delayed(get_df)(name) for name in file_names)

df_procedimientos = pd.concat(dfs, axis=0, ignore_index=True)
df_procedimientos = df_procedimientos.loc[(df_procedimientos.FECHA_INICIO.dt.year > 2011), :]

Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-6:


KeyboardInterrupt: 

In [75]:
c = CurrencyConverter(fallback_on_missing_rate=True)
df_procedimientos["IMPORTE_PESOS"] = df_procedimientos.IMPORTE_CONTRATO
for m in df_procedimientos.index:
    if df_procedimientos.MONEDA[m] not in ["MXN","TEST","OTH"]:
        df_procedimientos.loc[m,"IMPORTE_PESOS"] = c.convert(df_procedimientos.IMPORTE_CONTRATO[m],df_procedimientos.MONEDA[m],'MXN',date= df_procedimientos.FECHA_INICIO[m])

In [2]:
#df_procedimientos.to_csv("../data/procedimientos.csv",index=False)
df_procedimientos = pd.read_csv("../data/procedimientos.csv",low_memory=False)

In [3]:
df_procedimientos.RESPONSABLE = df_procedimientos.RESPONSABLE.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.lower()
df_procedimientos.DEPENDENCIA = df_procedimientos.DEPENDENCIA.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.lower()
df_procedimientos.PROVEEDOR_CONTRATISTA = df_procedimientos.PROVEEDOR_CONTRATISTA.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.lower()
df_procedimientos.RESPONSABLE = df_procedimientos.RESPONSABLE.str.replace('  '," ")
df_procedimientos.DEPENDENCIA = df_procedimientos.DEPENDENCIA.str.replace('  '," ")
df_procedimientos.PROVEEDOR_CONTRATISTA = df_procedimientos.PROVEEDOR_CONTRATISTA.str.replace('  '," ")

#df_procedimientos.DEPENDENCIA = df_procedimientos.DEPENDENCIA.str.replace("hospital general de mexico$",'hospital general de mexico \"dr. eduardo liceaga\"')
#df_procedimientos.DEPENDENCIA = df_procedimientos.DEPENDENCIA.str.replace('tribunal federal de justicia fiscal y administrativa',"tribunal federal de justicia administrativa")
#df_procedimientos.DEPENDENCIA = df_procedimientos.DEPENDENCIA.str.replace('instituto de investigaciones electricas',"instituto nacional de electricidad y energias limpias")
#df_procedimientos.DEPENDENCIA = df_procedimientos.DEPENDENCIA.str.replace('servicios aeroportuarios de la ciudad de mexico, s.a. de c.v.',"aeropuerto internacional de la ciudad de mexico, s.a. de c.v.")
#df_procedimientos.DEPENDENCIA = df_procedimientos.DEPENDENCIA.str.replace('seguros de credito a la vivienda shf, s.a. de c.v.','sociedad hipotecaria federal, s.n.c.')

In [ ]:
federales = df_procedimientos.loc[(df_procedimientos.GOBIERNO=="APF"),:]

In [57]:
contratos_dup = df_procedimientos.groupby(["CODIGO_CONTRATO","CODIGO_EXPEDIENTE"],as_index=False).agg({"CLAVEUC":"size"})
contratos_dup.columns = ["CODIGO_CONTRATO","CODIGO_EXPEDIENTE","nr_reportes_procedimientos"]

In [6]:
860101+526*2+4*4

861169

In [7]:
860101+526+4

860631

In [58]:
repetidos = pd.merge(contratos_dup.loc[contratos_dup.nr_reportes_procedimientos>1,:],df_procedimientos,on=["CODIGO_CONTRATO","CODIGO_EXPEDIENTE"],how="left")

In [ ]:
SG/CNS/98/2014

In [65]:
repetidos.loc[repetidos.GOBIERNO=="APF",:].drop_duplicates(keep=False)

,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,nr_reportes_procedimientos,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE,IMPORTE_PESOS
0,195917,230616,2,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Internacional,018TOQ028,NaN,NaN,0.0,0.0,0.0,PoC,0.0,comision federal de electricidad,Terminado,HABILITADO,Micro,No MIPYME,2012-09-18 00:00:00 GMT,2012-10-24 00:00:00 GMT,NaN,2012-12-30,2012-09-18,NaN,Electrónica,APF,NaN,5.995400e+04,MXN,CFE-Gerencia Divisional Noroeste #018TOQ028,LA-018TOQ028-I75-2012,NaN,Z15122015 03. Licitación Pública Internacional...,0.0,NaN,as tecnologias modernas sa de cv,NaN,hugo cesar sortillon gonzalez,CFE,MX,Adquisiciones,Licitación Pública,DDNO-022 Refacciones para cfematico,DDNO-022 Refacciones para cajeros automatico 2...,5.995400e+04
1,195917,230616,2,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Internacional,018TOQ028,NaN,NaN,0.0,0.0,0.0,PoC,0.0,comision federal de electricidad,Expirado,HABILITADO,Micro,No MIPYME,2012-09-18 00:00:00 GMT,2012-10-24 00:00:00 GMT,NaN,2017-02-28,2016-09-18,NaN,Electrónica,APF,NaN,5.629000e+04,MXN,CFE-Gerencia Divisional Noroeste #018TOQ028,LA-018TOQ028-I75-2012,NaN,Z15122015 03. Licitación Pública Internacional...,0.0,NaN,as tecnologias modernas sa de cv,NaN,hugo cesar sortillon gonzalez,CFE,MX,Adquisiciones,Licitación Pública,DDNO-022 Refacciones para cfematico,DDNO-022 Refacciones para cajeros automatico 2...,5.629000e+04
2,195918,230616,2,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Internacional,018TOQ028,NaN,NaN,0.0,0.0,0.0,PoC,0.0,comision federal de electricidad,Expirado,HABILITADO,Micro,No MIPYME,2012-09-18 00:00:00 GMT,2012-10-24 00:00:00 GMT,NaN,2012-12-12,2012-09-18,NaN,Electrónica,APF,NaN,8.046600e+04,MXN,CFE-Gerencia Divisional Noroeste #018TOQ028,LA-018TOQ028-I75-2012,NaN,Z15122015 03. Licitación Pública Internacional...,0.0,NaN,l y f ingenieria sa de cv,NaN,hugo cesar sortillon gonzalez,CFE,MX,Adquisiciones,Licitación Pública,DDNO-022 Refacciones para cfematico,DDNO-022 Refacciones para cajeros automatico 2...,8.046600e+04
3,195918,230616,2,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Internacional,018TOQ028,NaN,NaN,0.0,0.0,0.0,PoC,0.0,comision federal de electricidad,Terminado,HABILITADO,Micro,No MIPYME,2012-09-18 00:00:00 GMT,2012-10-24 00:00:00 GMT,NaN,2016-12-10,2016-09-18,NaN,Electrónica,APF,NaN,8.046600e+04,MXN,CFE-Gerencia Divisional Noroeste #018TOQ028,LA-018TOQ028-I75-2012,NaN,Z15122015 03. Licitación Pública Internacional...,0.0,NaN,l y f ingenieria sa de cv,NaN,hugo cesar sortillon gonzalez,CFE,MX,Adquisiciones,Licitación Pública,DDNO-022 Refacciones para cfematico,DDNO-022 Refacciones para cajeros automatico 2...,8.046600e+04
4,256016,226807,2,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Nacional,020VST001,NaN,NaN,0.0,0.0,1.0,PoC,NaN,"liconsa, s.a. de c.v.",Expirado,HABILITADO,Micro,Micro,NaN,NaN,2012-08-03 00:00:00 GMT,2016-01-12,2012-07-13,12733.0,Mixta,APF,NaN,6.600000e+05,MXN,LICONSA-Subdirección de Adquisiciones de Consu...,SA-020VST001-N125-2012,NaN,Z15122015 12. Adjudicación Directa Nacional Si...,0.0,NaN,profeti consultores sa de cv,NaN,othoniel melchor pena montor,LICONSA,MX,Servicios,Adjudicación Directa Federal,AMPLIACIÓN DEL 20% AL SERVICIOS DE MANTENIMIEN...,MANTENIMIENTO Y ADECUACION AL SISTEMA E-CONTAB...,6.600000e+05
5,256016,226807,2,NaN,NaN,No,Nacional,020VST001,NaN,NaN,0.0,0.0,NaN,PoC

In [6]:
gr_3 = federales.groupby(
    ["RESPONSABLE","DEPENDENCIA","ESTATUS_CONTRATO"],as_index=False).agg({"IMPORTE_PESOS":"sum","CODIGO_CONTRATO":"size"})
tabla = pd.pivot_table(data= gr_3,index=["RESPONSABLE","DEPENDENCIA"],columns=["ESTATUS_CONTRATO"],values=["CODIGO_CONTRATO","IMPORTE_PESOS"]).fillna(0).reset_index()
tabla.columns = ["RESPONSABLE","DEPENDENCIA","NR_ACTIVOS","NR_EXPIRADOS","NR_TERMINADOS","MONTO_ACTIVOS","MONTO_EXPIRADOS","MONTO_TERMINADOS"]
tabla["NR_TOTAL"] = tabla.loc[:,["NR_ACTIVOS","NR_EXPIRADOS","NR_TERMINADOS"]].sum(axis=1)
tabla["MONTO_TOTAL"] = tabla.loc[:,["MONTO_ACTIVOS","MONTO_EXPIRADOS","MONTO_TERMINADOS"]].sum(axis=1)


In [35]:
#federales.to_csv("../data/federales_tab.csv",sep="\t",index=False,na_rep="nan")

In [36]:
federales

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE,IMPORTE_PESOS
70,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,Si,Nacional,009JZL032,NaN,NaN,165782,115807,0.0,0.0,1.0,UC,NaN,aeropuertos y servicios auxiliares,Expirado,HABILITADO,NaN,Micro,2011-12-19 00:00:00 GMT,2011-12-16 00:00:00 GMT,NaN,2013-02-28,2012-01-01,NaN,Mixta,APF,NaN,319200.00,MXN,ASA-Estación de Combustibles del Aeropuerto de...,IA-009JZL032-N1-2012,NaN,Z15122015 04. Invitación a cuando menos tres N...,0.0,NaN,"multiservicios y asesorias del caribe, s.a. de...",NaN,jose luis almada penunuri,ASA,MX,Adquisiciones,Invitación a Cuando Menos 3 Personas,"Servicio de Limpieza, CZM 2012","Servicio de Limpieza, CZM 2012",319200.00
71,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,Si,Nacional,009JZL032,NaN,NaN,165738,242370,0.0,0.0,0.0,UC,NaN,aeropuertos y servicios auxiliares,Expirado,HABILITADO,NaN,Micro,NaN,2012-08-10 16:05,2012-08-20 00:00:00 GMT,2012-12-31,2012-08-26,NaN,Presencial,APF,NaN,26097.00,MXN,ASA-Estación de Combustibles del Aeropuerto de...,AA-009JZL032-N4-2012,NaN,Z15122015 07. Adjudicación Directa Nacional Ar...,0.0,2012-08-10 15:59,alejandro alarcon jimon,NaN,jose luis almada penunuri,ASA,MX,Servicios,Adjudicación Directa Federal,Mantenimiento a Extintores CZM,"Servicio de Mantenimiento a Extintores, AD CZM...",26097.00
72,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,Si,Nacional,009JZL032,NaN,NaN,148131,116667,0.0,0.0,1.0,PoC,NaN,aeropuertos y servicios auxiliares,Expirado,HABILITADO,Mediana,Micro,2011-12-19 00:00:00 GMT,2011-12-16 14:00,NaN,2013-02-28,2012-01-01,5578.0,Mixta,APF,NaN,361200.00,MXN,ASA-Estación de Combustibles del Aeropuerto de...,IA-009JZL032-N2-2012,NaN,Z15122015 04. Invitación a cuando menos tres N...,0.0,2011-12-09 14:51,transporte especializado de personal nacional ...,NaN,jose luis almada penunuri,ASA,MX,Servicios,Invitación a Cuando Menos 3 Personas,SERVICIO DE TRANSPORTE DE PERSONAL DE LA ESTAC...,"Servicio de Transporte de Personal, CZM 2012",361200.00
73,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,Si,Nacional,009JZL032,NaN,NaN,276283,123933,0.0,0.0,1.0,UC,NaN,aeropuertos y servicios auxiliares,Expirado,HABILITADO,NaN,Micro,NaN,2011-12-28 17:00,NaN,2013-02-28,2012-01-01,NaN,Mixta,APF,NaN,334264.00,MXN,ASA-Estación de Combustibles del Aeropuerto de...,AA-009JZL032-N5-2011,NaN,Z15122015 07. Adjudicación Directa Nacional Ar...,0.0,2011-12-28 12:45,"comercializadora el maharaja, s.a. de c.v.",NaN,jose luis almada penunuri,ASA,MX,Adquisiciones,Adjudicación Directa Federal,SERVICIO DE COMEDOR DEL PERSONAL DE LA ESTACIO...,Servicio de Comedor AD CZM 2012,334264.00
74,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,Si,Nacional,009JZL032,NaN,NaN,148083,123933,0.0,NaN,0.0,UC,NaN,aeropuertos y servicios auxiliares,Expirado,HABILITADO,NaN,Mediana,NaN,2011-12-28 17:00,2011-12-30 00:00:00 GMT,2012-12-31,2012-01-01,NaN,Mixta,APF,NaN,285948.00,MXN,ASA-Estación de Combustibles del Aeropuerto de...,AA-009JZL032-N5-2011,NaN,Z15122015 07. Adjudicación Directa Nacional Ar...,0.0,2011-12-28 12:45,"comercializadora el maharaja, s.a. de c.v.",NaN,jose luis almada penunuri,ASA,MX,Servicios,Adjudicación Directa Federal,SERVICIO DE COMEDOR PARA EL PERSONAL DE LA EST...,Servicio de Comedor AD CZM 2012,285948.00
75,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,Si,Nacional,009JZL032,N

In [55]:
df_procedimientos.loc[df_procedimientos.CODIGO_EXPEDIENTE==1292043,:]

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE,IMPORTE_PESOS


In [38]:
df_procedimientos.loc[df_procedimientos.PROVEEDOR_CONTRATISTA.notnull()&df_procedimientos.PROVEEDOR_CONTRATISTA.str.contains("opintel"),"PROVEEDOR_CONTRATISTA"]

Series([], Name: PROVEEDOR_CONTRATISTA, dtype: object)

In [25]:
df_procedimientos.loc[df_procedimientos.PROVEEDOR_CONTRATISTA.notnull()&df_procedimientos.PROVEEDOR_CONTRATISTA.str.contains("open intelligence"),"TITULO_CONTRATO"].values

array([' Elaboración de un Video Informativo Institucional'], dtype=object)

In [14]:
df_procedimientos.loc[df_procedimientos.PROVEEDOR_CONTRATISTA.isnull(),:]

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE,IMPORTE_PESOS
182035,NaN,0.00,No,Nacional,915005993,NaN,NaN,458682,505681,0.0,0.0,NaN,PoC,NaN,_gobierno del estado de mexico,Expirado,HABILITADO,Pequeña,No MIPYME,2013-12-02 00:00:00 GMT,2013-11-29 10:00:00,2013-12-12 00:00:00 GMT,2014-12-12,2013-12-12,NaN,Presencial (Estatal),GE,NaN,151200.00,MXN,MEX-Secretaría de Educación -INSTITUTO MEXIQUE...,EA-915005993-N21-2013,NaN,28. Licitación Pública Nacional de Adquisicion...,0.0,2013-11-21 17:18:00,NaN,NaN,clara lilia mejia reyes,MEX,NaN,Adquisiciones,Licitación Pública,BIENES INFORMÁTICOS,BIENES INFORMÁTICOS,151200.00
182036,NaN,0.00,No,Nacional,915005993,NaN,NaN,458680,505681,0.0,0.0,NaN,PoC,NaN,_gobierno del estado de mexico,Expirado,HABILITADO,Pequeña,No MIPYME,2013-12-02 00:00:00 GMT,2013-11-29 10:00:00,2013-12-12 00:00:00 GMT,2014-12-12,2013-12-12,NaN,Presencial (Estatal),GE,NaN,1055880.00,MXN,MEX-Secretaría de Educación -INSTITUTO MEXIQUE...,EA-915005993-N21-2013,NaN,28. Licitación Pública Nacional de Adquisicion...,0.0,2013-11-21 17:18:00,NaN,NaN,clara lilia mejia reyes,MEX,NaN,Adquisiciones,Licitación Pública,BIENES INFORMÁTICOS,BIENES INFORMÁTICOS,1055880.00
184934,NaN,232076.64,Si,Nacional,9000967,7096400018,08U017-SIST NAL INF DES SUSTENTABLE,728661,479936,1.0,0.0,NaN,NaN,NaN,secretaria de comunicaciones y transportes,Expirado,NaN,NaN,NaN,2013-10-16 00:00:00 GMT,2013-10-11 00:00:00,2013-10-22 00:00:00 GMT,2013-12-15,2013-10-21,NaN,Presencial,APF,NaN,232076.64,MXN,SCT-Centro SCT Oaxaca Carreteras Alimentadoras,IO-009000967-N124-2013,NaN,19. Invitación a cuando menos tres Nacional LO...,0.0,NaN,NaN,9.0,jose daniel angeles ramirez,SCT,NaN,Servicios Relacionados con la OP,Invitación a Cuando Menos 3 Personas,SUPERVISION TLAXIACO,SUPERVISION TLAXIACO,232076.64
184939,NaN,516298.80,Si,Nacional,9000967,11096400009,08U017-SIST NAL INF DES SUSTENTABLE,728596,468080,1.0,0.0,NaN,NaN,NaN,secretaria de comunicaciones y transportes,Expirado,NaN,NaN,NaN,2013-09-30 00:00:00 GMT,2013-11-13 15:00:00,2013-10-04 00:00:00 GMT,2013-12-15,2013-10-04,NaN,Presencial,APF,NaN,516298.80,MXN,SCT-Centro SCT Oaxaca Carreteras Alimentadoras,IO-009000967-N123-2013,NaN,19. Invitación a cuando menos tres Nacional LO...,0.0,2013-11-13 14:42:00,NaN,9.0,jose daniel angeles ramirez,SCT,NaN,Servicios Relacionados con la OP,Invitación a Cuando Menos 3 Personas,SUPERVISION PUERTO,SUPERVISION PUERTO,516298.80
184941,NaN,4132972.18,Si,Nacional,9000967,12092100003,08U017-SIST NAL INF DES SUSTENTABLE,728571,468042,1.0,0.0,NaN,NaN,NaN,secretaria de comunicaciones y transportes,Expirado,NaN,NaN,NaN,2013-09-30 00:00:00 GMT,2013-11-13 13:00:00,2013-10-04 00:00:00 GMT,2013-11-30,2013-10-04,NaN,Presencial,APF,NaN,4132972.18,MXN,SCT-Centro SCT Oaxaca Carreteras Alimentadoras,IO-009000967-N120-2013,NaN,19. Invitación a cuando menos tres Nacional LO...,0.0,2013-11-12 19:20:00,NaN,9.0,jose daniel angeles ramirez,SCT,NaN,Obra Pública,Invitación a Cuando Menos 3 Personas,CONSERVACIÓN: KM 78+000 E.C. (HUAJUAPAN - OAXA...,CONSERVACIÓN: KM 78+000 E.C. (HUAJUAPAN - OAXA...,4132972.18
184942,NaN,3439149.54,Si,Nacional,9000967,12092100003,08U017-SIST NAL INF DES SUSTENTABLE,728555,468036,1.0,0.0,NaN,NaN,NaN,secretaria de comunicaciones y transportes,Expirado,NaN,NaN,NaN,2013-09-30 00:00:00 GMT,2013-11-13 12:00:00,2013-10-04 00:00:00 GMT,2013-11-30,2013-10-

In [408]:
#tabla.to_csv("../data/responsables.csv",sep="|",index=False)

In [494]:
tabla

,RESPONSABLE,DEPENDENCIA,NR_ACTIVOS,NR_EXPIRADOS,NR_TERMINADOS,MONTO_ACTIVOS,MONTO_EXPIRADOS,MONTO_TERMINADOS,NR_TOTAL,MONTO_TOTAL
0,aaron velazquez portas,pemex-gas y petroquimica basica,0.0,2.0,0.0,0.000000e+00,1.150870e+05,0.000000e+00,2.0,1.150870e+05
1,abel alejandro salgado rochin,servicio de administracion tributaria,0.0,25.0,33.0,0.000000e+00,3.758558e+06,3.513584e+06,58.0,7.272142e+06
2,abel cisneros gonzalez,fondo nacional de fomento al turismo,20.0,1021.0,1.0,1.265851e+08,3.206297e+09,1.750000e+05,1042.0,3.333057e+09
3,abel eliezer ramirez aguirre,comision federal de electricidad,0.0,231.0,99.0,0.000000e+00,1.121091e+08,3.236330e+07,330.0,1.444724e+08
4,abel perez triana,caminos y puentes federales de ingresos y serv...,0.0,30.0,0.0,0.000000e+00,1.497374e+08,0.000000e+00,30.0,1.497374e+08
5,abelardo placeres bezares,petroleos mexicanos (corporativo),2.0,160.0,0.0,2.129780e+07,1.927237e+08,0.000000e+00,162.0,2.140215e+08
6,abigail baizabal garcia,comision federal de electricidad,0.0,260.0,0.0,0.000000e+00,6.707049e+08,0.000000e+00,260.0,6.707049e+08
7,abraham chacha victorio,comision federal de electricidad,0.0,277.0,1.0,0.000000e+00,1.004581e+09,4.328317e+06,278.0,1.008910e+09
8,abraham obregon cerecer,centro nacional de programas preventivos y con...,0.0,311.0,0.0,0.000000e+00,1.586799e+09,0.000000e+00,311.0,1.586799e+09
9,abraham palacios ochoa,comision federal de electricidad,0.0,220.0,5.0,0.000000e+00,7.319174e+07,9.378400e+05,225.0,7.412958e+07


In [409]:
cambios = tabla.groupby("RESPONSABLE",as_index=False).agg({"DEPENDENCIA":"size"})
cambios = list(cambios.loc[cambios.DEPENDENCIA!=1,"RESPONSABLE"])

In [412]:
tabla.loc[tabla.RESPONSABLE.isin(cambios),:]

,RESPONSABLE,DEPENDENCIA,NR_ACTIVOS,NR_EXPIRADOS,NR_TERMINADOS,MONTO_ACTIVOS,MONTO_EXPIRADOS,MONTO_TERMINADOS,NR_TOTAL,MONTO_TOTAL
93,alfredo sandoval gonzalez,aeropuerto internacional de la ciudad de mexic...,31.0,931.0,0.0,1.190669e+09,3.676395e+09,0.000000e+00,962.0,4.867065e+09
94,alfredo sandoval gonzalez,servicios aeroportuarios de la ciudad de mexic...,1.0,23.0,0.0,4.540350e+05,1.510798e+07,0.000000e+00,24.0,1.556201e+07
103,alma delia sanchez ocampo,financiera nacional de desarrollo agropecuario...,1.0,251.0,1.0,1.901247e+07,7.220126e+08,1.795000e+05,253.0,7.412046e+08
104,alma delia sanchez ocampo,prevencion y readaptacion social,0.0,45.0,0.0,0.000000e+00,1.823136e+08,0.000000e+00,45.0,1.823136e+08
169,antonio orta rendon,hospital general de mexico,1.0,7.0,1.0,3.867404e+06,1.346446e+08,2.833555e+06,9.0,1.413455e+08
170,antonio orta rendon,"hospital general de mexico ""dr. eduardo liceaga""",0.0,7.0,11.0,0.000000e+00,1.645006e+08,2.236199e+08,18.0,3.881205e+08
183,armando eduardo gonzalez hernandez,instituto de investigaciones electricas,0.0,658.0,6.0,0.000000e+00,1.245682e+08,4.337029e+06,664.0,1.289052e+08
184,armando eduardo gonzalez hernandez,instituto nacional de electricidad y energias ...,21.0,2160.0,8.0,1.849420e+08,4.745461e+08,1.886852e+06,2189.0,6.613750e+08
234,bertha alicia robles quintero,"seguros de credito a la vivienda shf, s.a. de ...",1.0,28.0,0.0,2.561570e+05,5.648367e+06,0.000000e+00,29.0,5.904524e+06
235,bertha alicia robles quintero,"sociedad hipotecaria federal, s.n.c.",35.0,525.0,0.0,4.744353e+08,7.309313e+08,0.000000e+00,560.0,1.205367e+09


In [43]:
seguimiento = pd.read_excel("../data/raw/Anuncios en Seguimiento o Concluidos.xlsx",skiprows=6)

In [54]:
seguimiento.head()

,Código del Expediente,Descripción del Expediente,Referencia del Expediente,Tipo de Expediente,Descripción del Anuncio,Notas,Nombre de la Unidad Compradora (UC),Nombre de la Unidad Compradora (UC).1,Nombre del Operador en la UC,Correo Electrónico del Operador en la UC,Enlace Web,Tipo de Contratación,Entidad Federativa,Plazo de participación o vigencia del anuncio,Fecha de Inicio del Contrato,Duración del Contrato,Fecha estimada de conclusión del contrato,Categorias del Expediente,Número de Anexos,Avisos
0,1292043,"REHABILITACIÓN RED ALCANTARILLADO SANITARIO, C...",LO-801001996-E10-2017,02. Licitación Pública LOPSRM,"REHABILITACIÓN RED ALCANTARILLADO SANITARIO, C...",Notas Adicionales por Defecto,AGS-Aguascalientes-COMISION CIUDADANA DE AGUA ...,AGS-Aguascalientes-COMISION CIUDADANA DE AGUA ...,PAULO CESAR GONZALEZ AVILA,pgonzalez@ccapama.gob.mx,NaN,Obra Pública,Aguascalientes,21/03/2017,25/03/2017,37 días,30/04/2017,6130,0,0
1,1292015,"REHABILITACIÓN RED ALCANTARILLADO SANITARIO, F...",LO-801001996-E9-2017,02. Licitación Pública LOPSRM,"REHABILITACIÓN RED ALCANTARILLADO SANITARIO, F...",Notas Adicionales por Defecto,AGS-Aguascalientes-COMISION CIUDADANA DE AGUA ...,AGS-Aguascalientes-COMISION CIUDADANA DE AGUA ...,PAULO CESAR GONZALEZ AVILA,pgonzalez@ccapama.gob.mx,NaN,Obra Pública,Aguascalientes,21/03/2017,25/03/2017,37 días,30/04/2017,6130,0,0
2,1305881,AUT COMPRA MED REQ 1070 1072 4074 5071 5073 41 V,AA-019GYR032-E46-2017,05. Adjudicación Directa LAASSP,AUT COMPRA MED REQ 1070 1072 4074 5071 5073 41 V,Notas Adicionales por Defecto,IMSS-Coordinación de Abastecimiento y Equipami...,IMSS-Coordinación de Abastecimiento y Equipami...,Jose Manuel Hernandez Perez,jose.hernandezper@imss.gob.mx,NaN,Adquisiciones,Aguascalientes,21/03/2017,NaN,NaN,NaN,2530,1,0
3,1305887,I.T. EXTENSION DEL INSTITUTO TECNOLOGICO DE Z...,INV-SEV-IEEV-FED-001-17,04. Invitación a Cuando Menos Tres Personas LO...,TERMINACION DE CONSTRUCCION DE EDIFICIO ACADEM...,Notas Adicionales por Defecto,VER-Instituto de Espacios Educativos del Estad...,VER-Instituto de Espacios Educativos del Estad...,CARLOS FRANK JORGE MARTINEZ,fjorge@ifidev.gob.mx,NaN,Obra Pública,Veracruz,21/03/2017,31/03/2017,150,27-08-2017,6160,0,0
4,1305810,SERVICIO DE POLICIA AUXILIAR,"AA-019GYR055-E81-2017, PARA EL PERÍODO COMPREN...",05. Adjudicación Directa LAASSP,PARA EL PERÍODO COMPRENDIDO DEL 1 DE ABRIL AL ...,Notas Adicionales por Defecto,IMSS-Hospital de Especialidades Dr. Antonio Fr...,IMSS-Hospital de Especialidades Dr. Antonio Fr...,SAUL ORTIZ LOPEZ,saul.ortiz@imss.gob.mx,NaN,Servicios,Ciudad de México,21/03/2017,NaN,NaN,NaN,3380,1,0


In [52]:
PAAASOP_2013 = pd.read_csv("../data/raw/PAAASOP 2013.csv",encoding="latin1")

In [53]:
PAAASOP_2013.head()

,DEPENDENCIA_ENTIDAD,CLAVE_CUCOP,PRODUCTO,VALOR_ESTIMADO,VALOR_ MIPyMES,VALOR_NCTLC,CANTIDAD,UNIDAD_MEDIDA,CARACTER_PROCEDIMIENTO,ENTIDAD_FEDERATIVA,PORCENTAJE_TRIMESTRE1,PORCENTAJE_TRIMESTRE2,PORCENTAJE_TRIMESTRE3,PORCENTAJE_TRIMESTRE4,PLURIANUAL,AÑOS_PLURIANUALES,VALOR_ANUAL_PLURIANUAL,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION
0,Administración del Patrimonio de la Beneficenc...,29100142,Llave española,963.0,963.0,963.0,1.0,pieza,Nacional,Distrito Federal,0,100,0,0,No,0,0.0,Adjudicacion Directa,Adquisiciones
1,Administración del Patrimonio de la Beneficenc...,29100259,Segueta,102.0,102.0,102.0,8.0,pieza,Nacional,Distrito Federal,0,100,0,0,No,0,0.0,Adjudicacion Directa,Adquisiciones
2,Administración del Patrimonio de la Beneficenc...,29100099,Flexometro,143.0,143.0,143.0,2.0,pieza,Nacional,Distrito Federal,0,100,0,0,No,0,0.0,Adjudicacion Directa,Adquisiciones
3,Administración del Patrimonio de la Beneficenc...,29100223,Pinza punto pico,85.0,85.0,85.0,1.0,pieza,Nacional,Distrito Federal,0,100,0,0,No,0,0.0,Adjudicacion Directa,Adquisiciones
4,Administración del Patrimonio de la Beneficenc...,29100222,Pinza presion,149.0,149.0,149.0,1.0,pieza,Nacional,Distrito Federal,0,100,0,0,No,0,0.0,Adjudicacion Directa,Adquisiciones


In [489]:
por_periodo = federales.groupby(df_procedimientos.FECHA_INICIO.dt.year).agg({"CODIGO_CONTRATO":"size"}).astype(int)

In [490]:
por_periodo = por_periodo.reset_index()
por_periodo.columns = ["FECHA_INICIO","NR_TOTAL_CONTRATOS"]

In [493]:
por_periodo.to_csv("../data/por_periodo.csv",sep="|",index=False)

In [492]:
por_periodo

,FECHA_INICIO,NR_TOTAL_CONTRATOS
0,2012,125940
1,2013,120284
2,2014,144174
3,2015,172892
4,2016,178732


In [51]:
df_procedimientos.groupby(df_procedimientos.FECHA_INICIO.dt.year)["DEPENDENCIA"].nunique()

FECHA_INICIO
2012    302
2013    306
2014    318
2015    325
2016    321
Name: DEPENDENCIA, dtype: int64

In [55]:
df_procedimientos.groupby(df_procedimientos.FECHA_INICIO.dt.year).size()

FECHA_INICIO
2012    146500
2013    141912
2014    171691
2015    198956
2016    202110
dtype: int64

In [76]:
df_procedimientos.groupby(df_procedimientos.FECHA_FIN.dt.year).size()

FECHA_FIN
2012    130564
2013    135448
2014    168769
2015    203602
2016    206743
2017     13849
2018      1579
2019       503
2020        62
2021        24
2022         8
2023         2
2026         2
2031         5
2032         1
2035         1
2036         1
2039         2
2041         1
2045         1
2099         2
dtype: int64

In [50]:
df_procedimientos.groupby("TIPO_CONTRATACION")["IMPORTE_CONTRATO"].sum().astype(int)

TIPO_CONTRATACION
Adquisiciones                       698183401929
Arrendamientos                       19939963467
Obra Pública                        836962069415
Servicios                           581012685957
Servicios Relacionados con la OP     16287675879
Servicios relacionados con la OP     43158140256
Name: IMPORTE_CONTRATO, dtype: int64

In [49]:
estados = ["AGS","TLAX","DGO","MEX","MICH","QROO","VER","TAMPS","ZAC","SLP","YUC","NL","COL",
          "OAX","TAB","PUE","SON","JAL","QRO","COAH","BC","GTO","CHIH","GRO","CHIS","SIN","BCS",
           "MOR","CAMP","NAY","DF"]
contr_edos = df_procedimientos.loc[df_procedimientos.SIGLAS.isin(estados),:]
contr_edos.groupby(contr_edos.SIGLAS)["IMPORTE_CONTRATO"].sum().astype(int)

SIGLAS
AGS      10981234191
BC       12485848929
BCS       6847482381
CAMP      7025434033
CHIH      8960249080
CHIS     10284534612
COAH     15691544290
COL       1038230438
DF        3069166547
DGO      13616913860
GRO       5675539175
GTO      12472300068
JAL      13742302666
MEX      40210484366
MICH      8776174730
MOR       2944332843
NAY      15080019331
NL       61740600769
OAX      10984837459
PUE      28493666576
QRO      11601409059
QROO     16010917663
SIN      15447776499
SLP       5648507546
SON      22859152542
TAB       9069545246
TAMPS    13351931605
TLAX      4061330241
VER      24554995010
YUC       6695974410
ZAC       9325823480
Name: IMPORTE_CONTRATO, dtype: int64

In [72]:
df_procedimientos.loc[df_procedimientos.PROVEEDOR_CONTRATISTA.str.contains("Gusluher",case=False,na=False),:]

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE
550043,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Nacional,923012993,NaN,NaN,530538,576519,0.0,0.0,0.0,PoC,NaN,_Gobierno del Estado de Quintana Roo,Expirado,HABILITADO,Pequeña,No MIPYME,2014-04-15 00:00:00 GMT,2014-04-14 10:00,NaN,2016-02-29,2014-04-16,NaN,Mixta,GE,NaN,50360399.58,MXN,QROO-Oficialía Mayor-Dirección General de Admi...,IA-923012993-N7-2014,NaN,Z15122015 04. Invitación a cuando menos tres N...,0.0,2014-04-03 14:52,GUSLUHER ENVIROMENTAL SERVICE PROVIDERS SA DE CV,NaN,José Manuel González Acosta,QROO,MX,Adquisiciones,Invitación a Cuando Menos 3 Personas,Atlas Estatal de Riesgo del Estado de Quintana...,Atlas Estatal de Riesgo del Estado de Quintana...


In [75]:
df_procedimientos.loc[df_procedimientos.RESPONSABLE.str.contains("José Manuel González Acosta",case=False,na=False),:]

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE
550029,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Nacional,923012993,NaN,NaN,614117,685215,0.0,0.0,0.0,PoC,NaN,_Gobierno del Estado de Quintana Roo,Expirado,HABILITADO,Mediana,No MIPYME,NaN,NaN,NaN,2014-11-18,2014-10-02,NaN,Mixta,GE,NaN,18534482.76,MXN,QROO-Oficialía Mayor-Dirección General de Admi...,SA-923012993-N30-2014,NaN,Z15122015 12. Adjudicación Directa Nacional Si...,0.0,NaN,PROMOCION Y DESARROLLO DE EVENTOS CINEMATOGRAF...,NaN,José Manuel González Acosta,QROO,MX,Servicios,Adjudicación Directa Federal,Servicio integral de artistas Nacionales e Int...,Servicio integral de artistas Nacionales e Int...
550030,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Nacional,923012993,NaN,NaN,656040,719884,0.0,0.0,0.0,PoC,NaN,_Gobierno del Estado de Quintana Roo,Expirado,HABILITADO,No MIPYME,Pequeña,2014-11-28 00:00:00 GMT,2014-11-26 16:00,NaN,2014-12-29,2014-11-29,NaN,Mixta,GE,NaN,1284482.76,MXN,QROO-Oficialía Mayor-Dirección General de Admi...,IA-923012993-N44-2014,NaN,Z15122015 04. Invitación a cuando menos tres N...,0.0,2014-11-20 17:57,UNIVERSIDAD COOPERATIVA DE INNOVACION COMUNITA...,NaN,José Manuel González Acosta,QROO,MX,Servicios,Invitación a Cuando Menos 3 Personas,Diagnósticos participativos y de la violencia ...,Diagnósticos participativos y de la violencia ...
550031,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Internacional,923012993,NaN,NaN,639582,700262,0.0,0.0,0.0,PoC,0.0,_Gobierno del Estado de Quintana Roo,Expirado,HABILITADO,Mediana,No MIPYME,2014-11-04 00:00:00 GMT,2014-11-03 11:00,NaN,2014-12-03,2014-11-05,48931.0,Presencial,GE,NaN,576266.50,MXN,QROO-Oficialía Mayor-Dirección General de Admi...,EA-923012993-I34-2014,NaN,29. Licitación Pública Internacional de Adquis...,0.0,2014-10-24 20:37,NEOLOGY,NaN,José Manuel González Acosta,QROO,MX,Adquisiciones,Licitación Pública,Adquisción de Equipamiento para el Programa de...,Adquisción de Equipamiento para el Programa de...
550032,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Nacional,923012993,NaN,NaN,522042,586528,0.0,0.0,0.0,PoC,NaN,_Gobierno del Estado de Quintana Roo,Expirado,HABILITADO,Mediana,No MIPYME,2014-05-07 00:00:00 GMT,2014-05-06 10:00,NaN,2014-05-13,2014-05-08,70070.0,Presencial,GE,NaN,1719310.30,MXN,QROO-Oficialía Mayor-Dirección General de Admi...,EA-923012993-N9-2014,NaN,28. Licitación Pública Nacional de Adquisicion...,0.0,2014-04-23 13:55,CARIBE MOTORS SA DE CV,NaN,José Manuel González Acosta,QROO,MX,Adquisiciones,Licitación Pública,Adquisición de Vehículos,Adquisición de Vehículos
550033,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Nacional,923012993,NaN,NaN,620729,682553,0.0,0.0,0.0,PoC,NaN,_Gobierno del Estado de Quintana Roo,Expirado,HABILITADO,Pequeña,Pequeña,2014-10-07 00:00:00 GMT,2014-10-06 14:00,NaN,2014-12-10,2014-10-08,NaN,Mixta,GE,NaN,1668103.45,MXN,QROO-Oficialía Mayor-Dirección General de Admi...,IA-923012993-N28-2014,NaN,Z15122015 04. Invitación a cuando menos tres N...,0.0,2014-09-30 18:02,SUUT ICHIL KAAB: SERVICIOS ESTUDIOS Y PROYECTO...,NaN,José Manuel González Acosta,QROO,MX,Servicios,Invitación a Cuando Menos 3 Personas,"Eventos culturales, artísticos y deportivos y ...","Eventos culturales, artísticos y deportivos y ..."
550034,https://compranet.funcionpublica.gob.mx/esop/g...,NaN,No,Nacional,

In [77]:
sum(df_procedimientos.loc[df_procedimientos.RESPONSABLE.str.contains("José Manuel González Acosta",case=False,na=False),"IMPORTE_CONTRATO"])

293838284.04999995

In [67]:
df_procedimientos.loc[df_procedimientos.CODIGO_CONTRATO==208993,:]

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE
0,NaN,NaN,No,NaN,901024986,NaN,NaN,208993,271313,0.0,0.0,0.0,PoC,NaN,_Gobierno del Estado de Aguascalientes,Expirado,HABILITADO,Micro,No MIPYME,2012-10-23 00:00:00 GMT,2012-10-23 11:30,NaN,2012-12-31,2012-10-23,NaN,NaN,GE,NaN,512930.4,MXN,AGS-Instituto de Educación de Aguascalientes-I...,IA-901024986-N44-2012,NaN,Z15122015 04. Invitación a cuando menos tres N...,0.0,2012-10-17 14:51,RAUL EUGENIO ALVAREZ CARRASCO,NaN,Antonio Rubén Cienfuegos Benavides,AGS,MX,Adquisiciones,Invitación a Cuando Menos 3 Personas,N44 MOB ETC,ADQUISICIÓN DE MOBILIARIO


In [62]:
df_procedimientos.dtypes

ANUNCIO                                 object
APORTACION_FEDERAL                     float64
ARCHIVADO                               object
CARACTER                                object
CLAVEUC                                 object
CLAVE_CARTERA_SHCP                      object
CLAVE_PROGRAMA                          object
CODIGO_CONTRATO                          int64
CODIGO_EXPEDIENTE                        int64
COMPRA_CONSOLIDADA                     float64
CONTRATO_MARCO                         float64
CONVENIO_MODIFICATORIO                 float64
CUENTA_ADMINISTRADA_POR                 object
C_EXTERNO                              float64
DEPENDENCIA                             object
ESTATUS_CONTRATO                        object
ESTATUS_EMPRESA                         object
ESTRATIFICACION_MPC                     object
ESTRATIFICACION_MUC                     object
EXP_F_FALLO                             object
FECHA_APERTURA_PROPOSICIONES            object
FECHA_CELEBRA

In [37]:
df_procedimientos.loc[df_procedimientos.SIGLAS=="QRO","DEPENDENCIA"]

array(['AGS', 'ASA', 'IMSS', 'CFE', 'COMESA', 'TLAX', 'CENSIDA',
       'API-Ensenada', 'FOMMUR', 'SCT', 'DGO', 'ASERCA', 'MEX', 'CONEVAL',
       'MICH ', 'INEGI', 'CONAGUA', 'PGPB', 'PROSPERA', 'QROO', 'VER',
       'TAMPS', 'API-Veracruz', 'PPQ', 'ZAC', 'SLP', 'YUC', 'NL', 'COL',
       'OAX', 'PEMEX', 'PEP', 'SRE', 'IMER', 'TAB', 'PUE', 'IMP',
       'API-Lázaro Cárdenas', 'SEMARNAT', 'SON', 'CIMAV', 'LICONSA', 'QRO',
       'JAL', 'SEDATU', 'TELECOMM', 'ISSSTE', 'COAH', 'BC', 'GTO', 'CHIH',
       'INAH', 'INBAL', 'CAPUFE', 'SAT', 'API-Progreso', 'CIAD', 'CONAFE',
       'SAGARPA', 'SHCP', 'INM', 'COFEMER', 'CHIS', 'CONAPESCA', 'CDI',
       'CIDETEQ', 'DF', 'FIRCO', 'BANJERCITO', 'CONDUSEF', 'NAY', 'HGO',
       'DIF', 'NAFIN', 'CIMAT', 'SEDESOL', 'PREF', 'PRONOSTICOS', 'SEP',
       'API-Tuxpan', 'CINVESTAV', 'GRO', 'INCMNSZ', 'COLBACH', 'SIN',
       'CISEN', 'CAMP', 'EDUCAL', 'LOTENAL', 'CENACE', 'PR', 'INAES',
       'CONALEP', 'IMT', 'ININ', 'SEGOB', 'FND', 'SPF', 'INDAABIN'

In [85]:
pd.unique(df_procedimientos.DEPENDENCIA)

array(['_Gobierno del Estado de Aguascalientes',
       'Aeropuertos y Servicios Auxiliares',
       'Instituto Mexicano del Seguro Social',
       'Comisión Federal de Electricidad',
       'Compañía Mexicana de Exploraciones, S.A. de C.V.',
       '_Gobierno Municipal del Estado de Tlaxcala',
       'Centro Nacional para la Prevención y el Control del VIH/SIDA',
       'Administración Portuaria Integral de Ensenada, S.A. de C.V.',
       'Fideicomiso del Fondo de Microfinanciamiento a Mujeres Rurales',
       'Secretaría de Comunicaciones y Transportes',
       '_Gobierno Municipal del Estado de Durango',
       'Agencia de Servicios a la Comercialización y Desarrollo de Mercados Agropecuarios',
       '_Gobierno Municipal del Estado de México',
       'Consejo Nacional de Evaluación de la Política de Desarrollo Social',
       '_Gobierno Municipal del Estado de Michoacán de Ocampo',
       'Instituto Nacional de Estadística Y Geografía',
       'Comisión Nacional del Agua', 'Pemex-G